In [1]:

# mkconfig
import os
import shutil
from glob import glob

base_dict = {
    "name": "NAME"
    ,"version": "0.01"
    ,"license" : ""
    ,"setting_keywords":{
        "lang"         : "en"
        ,"font"         : "Meiryo UI"
        ,"width"        : "800"
        ,"height"       : "800"
    }
}
APPNAME = base_dict["name"]
WIDTH = base_dict["setting_keywords"]["width"]
HEIGHT = base_dict["setting_keywords"]["height"]

set_txt = ""
for key in list(base_dict["setting_keywords"].keys()):
    set_txt += f"""            ['{key}','{base_dict['setting_keywords'][key]}'],
"""

txt = f"""#!/usr/bin/env python3
import os
from {APPNAME}._config.config import config

class Conf(config):
    def __init__(self,name,**args):
        self.__name__ = "{APPNAME}"
        self.__version__ = "{base_dict["version"]}"
        self.__license__ = "{base_dict["license"]}"
        super().__init__(name=name,**args)
        default_set = [
{set_txt}
        ]
        self.set_log()
        for set_item in default_set:
            if self.get_data(set_item[0]) is None:
                self.set_data(set_item[0],set_item[1])

appconf = Conf(name=__name__)
"""

base_dir = os.path.join(os.getcwd(),APPNAME)
file_name = os.path.join(base_dir,"_util",'appconfig.py')
os.makedirs(os.path.dirname(file_name),exist_ok=True)
with open(file_name,"w",encoding="utf-8") as f:
    f.write(txt)

# 基礎ファイルの複製
base_files = glob(os.path.join(os.getcwd(),'base_files',"**","*"),recursive=True)
for file in base_files:
    dst_path = file.replace("base_files",APPNAME)
    if os.path.isfile(file) == False:
        os.makedirs(dst_path,exist_ok=True)
        continue
    os.makedirs(os.path.dirname(dst_path),exist_ok=True)
    shutil.copy(file,dst_path)

# __init__ file
file_name = os.path.join(os.getcwd(),APPNAME,"__init__.py")
with open(file_name,"w") as f:
    f.write("")
    


# create image.py
from glob import glob
import os
target = glob(os.path.join(base_dir,"img","*.png"))
target = [os.path.basename(t) for t in target]
img_py_file = os.path.join(base_dir,"_util","image.py")
base_txt = f"""#!/usr/bin/env python3
import customtkinter
import os
from PIL import Image

class ImageInst:
    def __init__(self) -> None:
        # load images with light and dark mode image
        image_path = os.path.join(os.getcwd(),"{APPNAME}", "img")"""

with open(img_py_file,"w",encoding="utf-8") as f:
    f.write(base_txt)
    for t in target:
        if t.find("image_")==0:
            if t.find("_light.")>-1:
                continue
            prefix_num = len( t.split("_")[0] ) + 1
            sufix_num = -( len( t.split("_")[-1] ) + 1 )
            key = t[prefix_num:][:sufix_num]
            tmp_txt = f"""
        self.image_{key} = customtkinter.CTkImage(light_image=Image.open(os.path.join(image_path, "image_{key}_dark.png")),
            dark_image=Image.open(os.path.join(image_path, "image_{key}_light.png")), size=(20, 20))"""
            f.write(tmp_txt)
        else:
            sufix_num = -( len( t.split(".")[-1] ) + 1 )
            key = t[:sufix_num]
            tmp_txt = f"""
        self.image_icon_{key} = customtkinter.CTkImage(Image.open(os.path.join(image_path, "{key}.png")), size=(20, 20))"""
            f.write(tmp_txt)
    tmp_txt = """
imginst = ImageInst()
"""
    f.write(tmp_txt)


# create main

txt = f"""#!/usr/bin/env python3
from {APPNAME}.main_menu import Mainmenu

def main():
    mm = Mainmenu()

if __name__ == "__main__":
    main()
"""

file_name = os.path.join(base_dir,'main.py')
with open(file_name,"w",encoding="utf-8") as f:
    f.write(txt)


# create 

txt = f"""
import json
import os
from glob import glob

class AppText:
    def __init__(self,current_language):
        self.translations = self.load_translations(current_language)
    
    def load_translations(self,language_code):
        with open(os.path.join(os.getcwd(),"{APPNAME}","locales",language_code + ".json"), 'r', encoding='utf-8') as f:
            return json.load(f)
    
    def translate(self,text):
        return self.translations.get(text, text)
    
    def lang_list(self):
        file_path = glob(os.path.join(os.getcwd(),"{APPNAME}","locales","*.json"))
        file_path = [os.path.basename(f) for f in file_path]
        file_path = [f.split(".")[0] for f in file_path]
        return file_path
"""

file_name = os.path.join(base_dir,'_util','apptext.py')
with open(file_name,"w",encoding="utf-8") as f:
    f.write(txt)


# mk locale
locale_dict = {
    "TEMPLATE" : {
        "en"    : ""
        ,"ja"   : ""
    }
    ,"APPNAME" : {
        "en"    : "AppName"
        ,"ja"   : "アプリ名"
    }
    ,"keyword" : {
        "en"    : "Keyword"
        ,"ja"   : "キーワード"
    }
    , "Menu1" : {
        "en"    : "Menu1"
        ,"ja"   : "メニュー1"
    }
    ,"Menu2" : {
        "en"    : "Menu2"
        ,"ja"   : "メニュー2"
    }
    ,"Menu3" : {
        "en"    : "Menu3"
        ,"ja"   : "メニュー3"
    }
    ,"About" : {
        "en"    : "About"
        ,"ja"   : "アプリについて"
    }
    ,"readme" : {
        "en"    : "Readme"
        ,"ja"   : "説明"
    }
    ,"version" : {
        "en"    : "Versino"
        ,"ja"   : "バージョン"
    }
}

old_files = glob(os.path.join(base_dir,"locales","*"))
for old_file in old_files:
    os.remove(old_file)

for num,key in enumerate(list(locale_dict.keys())):
    for lang in list(locale_dict[key].keys()):
        tmp_file_path = os.path.join(base_dir,"locales",f'{lang}.json')
        os.makedirs(os.path.dirname(tmp_file_path),exist_ok=True)
        if os.path.exists(tmp_file_path)==False:
            with open(tmp_file_path,"a",encoding="utf-8") as f:
                f.write("{\n")
        with open(tmp_file_path,"a",encoding="utf-8") as f:
            tmp_line = f'    "{key}": "{locale_dict[key][lang]}"'
            if len(list(locale_dict.keys())) != (num + 1):
                tmp_line += ",\n"
            else:
                tmp_line += "\n}"
            f.write(tmp_line)



In [2]:
# create main_menu

# from tkinter import filedialog

main_menu_dict = {
    "Menu1" : {
        "MeinSetting" : {
            "border_color"       : "theme.back2"
            ,"bg_color"          : '"transparent"'
            ,"hover_color"       : "[theme.back1, theme.highlight]"
            ,"fg_color"          : "[theme.back2,theme.back1]"
            ,"text_color"        : "[ theme.font_color2 , theme.font_color1]"
        }
        ,"dropdown" : {
            
        }
    }
    ,"Menu2" : {
        "MeinSetting" : {
            "border_color"       : "theme.back2"
            ,"bg_color"          : '"transparent"'
            ,"hover_color"       : "[theme.back1, theme.highlight]"
            ,"fg_color"          : "[theme.back2,theme.back1]"
            ,"text_color"        : "[ theme.font_color2 , theme.font_color1]"
        }
    }
    ,"Menu3" : {
        "MeinSetting" : {
            "border_color"       : "theme.back2"
            ,"bg_color"          : '"transparent"'
            ,"hover_color"       : "[theme.back1, theme.highlight]"
            ,"fg_color"          : "[theme.back2,theme.back1]"
            ,"text_color"        : "[ theme.font_color2 , theme.font_color1]"
        }
        ,"dropdown" : {
            "menu3_1" : {
                "action" : "ThemeFrame"
                ,"CmainFlame" : "2"
                ,"cell_size" : "6"
                ,"Frameitems" : [
                    
                ]
            }
            ,"menu3_2" : {
                "action" : "ThemeScrollableFrame"
                ,"CmainFlame" : "B2"
                ,"cell_size" : "6"
                ,"Frameitems" : [
                    
                ]
            }
        }
    }
    ,"Setting" : {
        "MeinSetting" : {
            "border_color"       : "theme.back2"
            ,"bg_color"          : '"transparent"'
            ,"hover_color"       : "[theme.back1, theme.highlight]"
            ,"fg_color"          : "[theme.back2,theme.back1]"
            ,"text_color"        : "[ theme.font_color2 , theme.font_color1]"
        }
        ,"dropdown" : {
            "edit" : {
                "action" : "ThemeFrame"
                ,"CmainFlame" : "2"
                ,"cell_size" : "6"
                ,"Frameitems" : [
                    "title" : {
                        
                    }
                    ,"" : {
                        
                    }
                ]
            }
            ,"load" : {
                "action" : "ThemeFrame"
                ,"CmainFlame" : "1"
                ,"cell_size" : "6"
                ,"Frameitems" : [
                    
                ]
            }
        }
    }
    ,"About" : {
        "MeinSetting" : {
            "border_color"       : "theme.back2"
            ,"bg_color"          : '"transparent"'
            ,"hover_color"       : "[theme.back1, theme.highlight]"
            ,"fg_color"          : "[theme.back2,theme.back1]"
            ,"text_color"        : "[theme.font_color2 ,theme.font_color1]"
        }
        ,"dropdown" : {
            "readme" : {
                "action" : "Script"
                ,"script": """        file_path = os.path.join(os.getcwd(),"readme.pdf")
        import webbrowser
        webbrowser.open(file_path)
"""
            }
            ,"version" : {
                "action" : "Script"
                ,"script": """        version_txt = f\"\"\"
AppName:    {appconf.__name__}
Version:    {appconf.__version__}
License:    {appconf.__license__}
\"\"\"
        show_info(title= AppText(appconf.get_data("version")),message=version_txt)
"""
            }
        }
    }
}


txt = f"""#!/usr/bin/env python3
import os
import customtkinter
from {APPNAME}.CTkMenuBar import *
from {APPNAME}._config.messagebox import *
from {APPNAME}._util.theme import *
from {APPNAME}._util.image import imginst
from {APPNAME}._util.apptext import AppText
from {APPNAME}._util.appconfig import appconf

"""
# import 追加
for key in list(main_menu_dict.keys()):
    if "dropdown" not in list(main_menu_dict[key].keys()):
        continue
    for sub_key in list(main_menu_dict[key]["dropdown"].keys()):
        if main_menu_dict[key]["dropdown"][sub_key]["action"] == "Script":
            continue
        os.makedirs(os.path.join(base_dir,key),exist_ok=True)
        with open(os.path.join(base_dir,key,"__init__.py"),"w") as f:
            f.write("")
        txt += f"""from {APPNAME}.{key}.{sub_key} import C{sub_key}
"""

txt += f"""

class Mainmenu(customtkinter.CTk):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.apptxt = AppText(appconf.get_data("lang"))
        self.call_mein_menu()
    
    @appconf.log_exception
    def call_mein_menu(self):
        self.width = appconf.get_data("width")
        self.height = appconf.get_data("height")
        self.geometry(f"{WIDTH}x{HEIGHT}")
        self.title(self.apptxt.translate("APPNAME"))
        theme = ThemeManager()
        customtkinter.set_appearance_mode(theme.mode)
        self.font = theme.setfont
        
        self.menu = CTkMenuBar(self)
"""

for key in list(main_menu_dict.keys()):
    txt += f"""        self.{key} = self.menu.add_cascade(self.apptxt.translate("{key}"))
"""

for key in list(main_menu_dict.keys()):
    txt += f"""        self.{key}_dropdown = CustomDropdownMenu(
            widget          = self.{key},
            font            = self.font,
"""
    for items in ["border_color","bg_color","hover_color","fg_color","text_color"]:
        if items in list(main_menu_dict[key]["MeinSetting"].keys()):
            txt += f"""            {items}    = {main_menu_dict[key]["MeinSetting"][items]},
"""
    txt += """            )
"""
    if "dropdown" not in list(main_menu_dict[key].keys()):
        continue
    for sub_key in list(main_menu_dict[key]["dropdown"].keys()):
        txt += f"""        self.{key}_dropdown.add_option(option=self.apptxt.translate("{sub_key}"),command=self.call_{sub_key})
"""

txt += f"""        self.menuframe = ThemeFrame1(
            master=self,
            fg_color = theme.back1,
            width = self.width,
            height = self.height,
            )
        self.menuframe.pack(fill="both")
        self.mainloop()
        
"""


for key in list(main_menu_dict.keys()):
    if "dropdown" not in list(main_menu_dict[key].keys()):
        continue
    for sub_key in list(main_menu_dict[key]["dropdown"].keys()):
        txt += f"""    @appconf.log_exception
    def call_{sub_key}(self):
"""
        if main_menu_dict[key]["dropdown"][sub_key]["action"] == "Script":
            txt += main_menu_dict[key]["dropdown"][sub_key]["script"]
        elif main_menu_dict[key]["dropdown"][sub_key]["action"] == "ThemeFrame" or main_menu_dict[key]["dropdown"][sub_key]["action"] == "ThemeScrollableFrame":
            flame_name_line = main_menu_dict[key]["dropdown"][sub_key]["action"] + main_menu_dict[key]["dropdown"][sub_key]["CmainFlame"].replace("B","Bold")
            txt += f"""        self.reset_frame()
        self.{sub_key} = C{sub_key}(self.menuframe)
        
"""
            class_file_name = os.path.join(base_dir,key,f"{sub_key}.py")
            cell_size = main_menu_dict[key]["dropdown"][sub_key]["cell_size"]
            ctxt = f"""#!/usr/bin/env python3

import os
import pyautogui
import tkinter as tk
import customtkinter
from pathlib import Path
from {APPNAME}._util.theme import *
from {APPNAME}._util.image import imginst
from {APPNAME}._util.apptext import AppText
from {APPNAME}._util.appconfig import appconf

class C{sub_key}({flame_name_line}):
    def __init__(
        self,
        master,
        **kwargs):
        self.trans = AppText(appconf.get_data("lang"))
        self.width = appconf.get_data("width")
        self.height = appconf.get_data("height")
        self.padx = 5
        self.pady = 5
        self.cell_max = {cell_size}
        self.cell = self.width / self.cell_max - self.padx * 2
        super().__init__(master,
            width                           = self.width
            ,height                         = self.height
            ,corner_radius                  = 0           
            )
        
        self.grid(row=0, column=0, sticky="nw")
        self.call_item_frame()
    
    def call_item_frame(self):
        row_i = 0
        """
            with open(class_file_name,"w",encoding="utf-8") as f:
                f.write(ctxt)

txt += f"""    def reset_frame(self):
        theme = ThemeManager()
        self.menuframe.destroy()
        self.menuframe = ThemeFrame1(
            master=self,
            fg_color = theme.back1,
            width = self.width,
            height = self.height,
            )
        self.menuframe.pack(fill="both")
"""


file_name = os.path.join(base_dir,'main_menu.py')
with open(file_name,"w",encoding="utf-8") as f:
    f.write(txt)


In [ ]:
# run
from NAME.main import main
main()

[ ERROR 2024-10-29 20:12:12,911] unsupported operand type(s) for +: 'NoneType' and 'str'
Traceback (most recent call last):
  File "c:\develop\python\CtkinterAppCreater\NAME\_config\config.py", line 127, in wrapper
    return func(*args, **kwargs)
  File "c:\develop\python\CtkinterAppCreater\NAME\main_menu.py", line 131, in call_version
    show_info(title= AppText(appconf.get_data("version")),message=version_txt)
  File "c:\develop\python\CtkinterAppCreater\NAME\_util\apptext.py", line 8, in __init__
    self.translations = self.load_translations(current_language)
  File "c:\develop\python\CtkinterAppCreater\NAME\_util\apptext.py", line 11, in load_translations
    with open(os.path.join(os.getcwd(),"NAME","locales",language_code + ".json"), 'r', encoding='utf-8') as f:
TypeError: unsupported operand type(s) for +: 'NoneType' and 'str'

